## Import Libraries

In [11]:
from time import sleep

In [12]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [13]:
import matplotlib.pyplot as plt
plt.ion()

In [14]:
from pydgilib_extra import *

Set the path to `dgilib.dll`.

In [15]:
dgilib_path = "C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Python\\dgilib.dll"

## Data Logging

Create a figure for the plot.

In [16]:
fig = plt.figure(figsize=(10, 6))
fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [17]:
config_dict = {
    "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
    "power_buffers": [{"channel": CHANNEL_A, "power_type": POWER_CURRENT}],
    "read_mode": [True, True, True, True],
    "write_mode": [False, False, False, False],
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "gpio_delay_time" : 0.00075,
    "augment_gpio": True,
    "fig": fig,
    "verbose": 0,
}

Perform the measurement.

In [19]:
data = []
fig.clf()

with DGILibExtra(dgilib_path, **config_dict) as dgilib:
    dgilib.device_reset()
    dgilib.logger.start()
    sleep(1)
    dgilib.logger.update_callback()
    print(dgilib.data)
    while not all(dgilib.data[INTERFACE_GPIO][-1]):
        dgilib.logger.update_callback()
#         sleep(1)
    sleep(0.1)
    dgilib.logger.stop()
    data = dgilib.data

Interfaces:
	 256:   power,   samples:   15000
	  48:   gpio,    samples:     218



## Analysis

In [ ]:
nsc_store_charge, nsc_store_time = power_and_time_per_pulse(data, 2)

In [ ]:
nsc_load_charge, nsc_load_time = power_and_time_per_pulse(data, 3)

In [ ]:
MIN_NUM_BYTES = 1
num_bytes = range(MIN_NUM_BYTES, MIN_NUM_BYTES + len(nsc_store_charge))
print(f"MAX_NUM_BYTES: {MIN_NUM_BYTES + len(nsc_store_charge) - 1}")

In [ ]:
from lmfit import Model

def line(x, slope, intercept):
    """a line"""
    return [slope*i + intercept for i in x]

mod = Model(line)
pars = mod.make_params(slope=0, intercept=1)

In [ ]:
results = []
for y in [nsc_store_charge, nsc_load_charge, nsc_store_time, nsc_load_time]:
    result = mod.fit(y, pars, x=num_bytes)
    print(result.fit_report())
    results.append(result)

In [ ]:
fig2 = plt.figure(figsize=(9, 8))

In [ ]:
charge_color = 'r'
time_color = 'b'

In [ ]:
fig2.clf()
# fig2.suptitle("Energy analysis of AES")
ax1 = fig2.add_subplot(1, 1, 1)
ax2 = ax1.twinx()
ax1.set_xlabel('Number of bytes')
ax1.set_ylabel('Charge [mC]', color=charge_color)
ax2.set_ylabel('Time [s]', color=time_color)
ax1.tick_params('y', colors=charge_color)
ax2.tick_params('y', colors=time_color)

In [ ]:
lines = []
lines += ax1.plot(num_bytes, nsc_store_charge, charge_color+'-', label='TrustZone Store Charge')
lines += ax1.plot(num_bytes, nsc_load_charge, charge_color+'--', label='TrustZone Load Charge')
lines += ax2.plot(num_bytes, nsc_store_time, time_color+'-', label='TrustZone Store Time')
lines += ax2.plot(num_bytes, nsc_load_time, time_color+'--', label='TrustZone Load Time')
ax1.legend(handles=lines)
# [nsc_store_charge, nsc_load_charge, nsc_store_time, nsc_load_time]
ax1.set_title(f"TrustZone Store Charge: Base {results[0].params['intercept'].value:.03} mC plus {results[0].params['slope'].value:.03} mC per byte\n" +
             f"TrustZone Load Charge: Base {results[1].params['intercept'].value:.03} mC plus {results[1].params['slope'].value:.03} mC per byte\n" +
             f"TrustZone Store Time: Base {results[2].params['intercept'].value:.03} s plus {results[2].params['slope'].value:.03} s per byte\n" +
             f"TrustZone Load Time: Base {results[3].params['intercept'].value:.03} s plus {results[3].params['slope'].value:.03} s per byte\n")
fig2.tight_layout()
fig2.show()

In [20]:
bool(InterfaceData())

False

In [21]:
bool(InterfaceData(1,2))

True

In [22]:
bool(LoggerData())

True

In [23]:
bool(LoggerData([INTERFACE_GPIO]))

True

In [24]:
data = LoggerData([INTERFACE_GPIO])

In [25]:
bool(data.gpio)

False

In [27]:
data.gpio += ([1],[2])

In [28]:
bool(data.gpio)

True

In [33]:
data = LoggerData()

In [30]:
bool(data)

False

In [31]:
data[3]

KeyError: 3

In [34]:
for key in data.keys():
    print(key)

48
256


In [35]:
dict(data)

{48: <pydgilib_extra.dgilib_data.InterfaceData at 0x11d9270>,
 256: <pydgilib_extra.dgilib_data.InterfaceData at 0x11d99f0>}

In [36]:
print(data)

Interfaces:
	  48:   gpio,    samples:       0
	 256:   power,   samples:       0

